<a href="https://colab.research.google.com/github/Burgunthy/Tensorflow2_Practice/blob/main/tf2-11-1-mnist_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# MNist를 사용하여 실습 진행
# softmax를 이용하여 output 구함
# layer의 크기를 바꿔가며 가장 효율적인 모델 구축 -> conv2d 사용
# loss: 0.0112 - accuracy: 0.9969
# 지금까지는 최고의 효율 출력
# 해당 공부를 통해 추후 UNET 등 tf2로 구현 가능

import numpy as np
import random
import tensorflow as tf

In [27]:
learning_rate = 0.001
batch_size = 100
# 한번에 100 개의 dataset 씩 학습
training_epochs = 15
# 15번의 training
nb_classes = 10
# 마지막 softmax 출력 값

drop_rate = 0.3
# drop out 함수 추가

In [28]:
mnist = tf.keras.datasets.mnist
# keras를 통해 mnist 파일 불러오기

(x_train, y_train), (x_test, y_test) = mnist.load_data()
# MNist data load. -> 추후 개인 공부에도 사용

x_train, x_test = x_train / 255.0, x_test / 255.0
# 픽셀을 255로 나눠 0 ~ 1 사이의 값으로 normalization

print(x_train.shape)  # (60000, 28, 28)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
# 이미지 자체를 입력하기 위해 채널을 추가해 28 * 28 * 1 로 reshape한다.

print(x_train.shape)

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
# in tf1, one_hot= True in read_data_sets("MNIST_data/", one_hot=True)
# 직접 one_hot encoding하여 train, test의 결과값을 저장한다

(60000, 28, 28)
(60000, 28, 28, 1)


In [29]:
tf.model = tf.keras.Sequential()
# 아래와 같은 선언에 익숙해져야 한다

tf.model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu'))
tf.model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
# L1 : CONV, POOL
# 16개의 필터를 사용한다. kernel size : 3 x 3

tf.model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
tf.model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
# L2
# 32개의 필터를 사용한다. kernel size : 3 x 3

tf.model.add(tf.keras.layers.Flatten())
tf.model.add(tf.keras.layers.Dense(units=10, kernel_initializer='glorot_normal', activation='softmax'))
# L3 fully connected -> 모든 수를 일자로 만들어 10 output_shape의 softmax를 진행한다

tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=learning_rate), metrics=['accuracy'])
tf.model.summary()

history =  tf.model.fit(x_train, y_train, batch_size=batch_size, epochs=training_epochs)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)                8010      
Total params: 12,810
Trainable params: 12,810
Non-trainable params: 0
__________________________________________________

In [30]:
predictions = tf.model.predict(x_test)
print('Prediction: \n', predictions)

score = tf.model.evaluate(x_train, y_train)
print('Accuracy: ', score[1])
# loss: 0.2455 - accuracy: 0.9320

Prediction: 
 [[7.0002421e-11 1.7072654e-09 8.3643738e-07 ... 9.9998355e-01
  2.5295257e-09 2.4048560e-07]
 [3.5516818e-09 1.0456401e-06 9.9999845e-01 ... 2.2816225e-11
  1.9648795e-07 3.4865336e-14]
 [1.8259490e-08 9.9988139e-01 3.9078823e-06 ... 3.0907951e-05
  7.6107840e-06 1.5827631e-07]
 ...
 [9.2958258e-13 1.9480494e-10 4.1317693e-13 ... 1.2233644e-07
  8.5813312e-08 1.6527505e-08]
 [2.3603977e-09 2.4970385e-15 1.3366920e-13 ... 4.8116693e-12
  2.1896930e-04 2.3106311e-10]
 [3.7317091e-10 7.7527367e-15 4.5774060e-09 ... 2.9328334e-12
  4.0867754e-08 2.2500491e-13]]
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0120 - accuracy: 0.9968
Accuracy:  0.9967666864395142


In [31]:
# test 하는 함수
random.seed(777)

y_predicted = tf.model.predict(x_test)
# x_test를 통해 미리 predict된 list 생성
for x in range(0, 10):
    random_index = random.randint(0, x_test.shape[0]-1)
    # x_test의 크기 중 random index를 구한다.
    
    print("index: ", random_index,
          "actual y: ", np.argmax(y_test[random_index]),
          "predicted y: ", np.argmax(y_predicted[random_index]))

index:  3757 actual y:  8 predicted y:  8
index:  7304 actual y:  5 predicted y:  5
index:  7300 actual y:  7 predicted y:  7
index:  6039 actual y:  9 predicted y:  9
index:  9429 actual y:  3 predicted y:  3
index:  4420 actual y:  5 predicted y:  5
index:  5507 actual y:  2 predicted y:  2
index:  8809 actual y:  1 predicted y:  1
index:  654 actual y:  5 predicted y:  5
index:  7302 actual y:  8 predicted y:  8
